<a href="https://colab.research.google.com/github/AlexXPZhu/XMUM-FYP-Code/blob/main/FYP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 Data cleaning
The dataset consists of 3 csv files, some of which are not in a good form.

lets firstly explre the *sqil.csv*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

## Here, you can change the name of file to explore different files.
# 导入数据
file_path = '/content/drive/MyDrive/FYP/merged_data.csv'
df = pd.read_csv(file_path)

# 看看长什么样子
display(df.head(10))

,Sentence,Label
0,a,1
1,a',1
2,a' --,1
3,a' or 1 = 1; --,1
4,@,1
5,?,1
6,' and 1 = 0 ) union all,1
7,? or 1 = 1 --,1
8,x' and userid is NULL; --,1
9,x' and email is NULL; --,1


In [ ]:
# 这一步，对数据做一些基本的了解
# Check data types
print(df.info())

# Check the distribution of the 'Label' column
print("\nLabel distribution:")
display(df['Label'].value_counts())

# Check for missing values
print("\nMissing values:")
display(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56897 entries, 0 to 56896
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  56897 non-null  object
 1   Label     56897 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 889.1+ KB
None

Label distribution:


,count
Label,
0,34633
1,22264



Missing values:


,0
Sentence,0
Label,0


we found that there are some missing value in the sentences, we can have a look of them to see if we need to drop them.

对于空的部分进行检查，看看能不能删除

In [ ]:
# Display rows where the 'Sentence' column is null
missing_sentences_df = df[df['Sentence'].isnull()]
display(missing_sentences_df)

,Sentence,Label


it looks that we should simply drop the null rows

看来是可以删除。（同样的一个内容，却对应着不同的lable）

In [ ]:
# Drop rows with missing values
df.dropna(inplace=True)

# Verify that missing values have been removed
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56897 entries, 0 to 56896
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  56897 non-null  object
 1   Label     56897 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 889.1+ KB
None


we can also remove duplicate rows

将重复的行也删除

In [ ]:
# Check for duplicate rows based on 'Sentence' and 'Label'
print("Number of duplicate rows:", df.duplicated(subset=['Sentence', 'Label']).sum())

# Drop duplicate rows based on 'Sentence' and 'Label'
df.drop_duplicates(subset=['Sentence', 'Label'], inplace=True)

# Verify that duplicates have been removed
print("\nNumber of rows after dropping duplicates:", len(df))

Number of duplicate rows: 0

Number of rows after dropping duplicates: 56897


*After processing, the availabile number of data for training is 33727*

最后，除掉重复和空的内容，可以用来训练的数据量是33727

#2 prepare for training

###2.1 TF-IDF & Logistic Regression

## Split data

将数据分割成训练集和测试集，使用train_test_split() 函数


In [ ]:
from sklearn.model_selection import train_test_split

X = df['Sentence']
y = df['Label']

# 对于数据进行分割
# Split the data
# random_state 是种子，保证了代码的可复现性
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True )

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (45517,)
Shape of X_test: (11380,)
Shape of y_train: (45517,)
Shape of y_test: (11380,)


## Tf-idf vectorization 向量化

### Subtask:
Apply TF-IDF vectorization to the 'Sentence' column to convert the text data into numerical features.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# 对数据进行向量化处理
# Fit and transform the training data
# 处理训练数据
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the testing data
# 处理测试数据
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("Shape of X_train_tfidf:", X_train_tfidf.shape)
print("Shape of X_test_tfidf:", X_test_tfidf.shape)

Shape of X_train_tfidf: (45517, 38697)
Shape of X_test_tfidf: (11380, 38697)


## Train logistic regression model

### Subtask:
Train a Logistic Regression model on the TF-IDF transformed training data.

使用逻辑回归模型

In [ ]:
from sklearn.linear_model import LogisticRegression

# Instantiate LogisticRegression model 模型初始化
lr_model = LogisticRegression()

# Train the model 训练模型
lr_model.fit(X_train_tfidf, y_train)

LogisticRegression()

## Evaluate model

### Subtask:
Evaluate the performance of the trained model on the test data using appropriate metrics.

对于模型进行评估，有四个指标

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test data
# 在测试数据上预测
y_pred = lr_model.predict(X_test_tfidf)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

display(y_pred)

Accuracy: 0.9760
Precision: 0.9866
Recall: 0.9519
F1-score: 0.9689


array([1, 0, 1, ..., 0, 1, 1])

In [ ]:
import numpy as np

# Count the occurrences of 0s and 1s in y_pred
counts = np.bincount(y_pred.astype(int))

# Print the counts
print("Count of 0s in y_pred:", counts[0] if len(counts) > 0 else 0)
print("Count of 1s in y_pred:", counts[1] if len(counts) > 1 else 0)

Count of 0s in y_pred: 7066
Count of 1s in y_pred: 4314


TF-IDF +LR on sqli.csv

1. Accuracy: 0.9620
2. Precision: 1.0000
3. Recall: 0.8529
4. F1-score: 0.9206

TF-IDF + LR on sqliv2.csv

1. Accuracy: 0.9846
2. Precision: 0.9973
3. Recall: 0.9580
4. F1-score: 0.9773

on merged_data
1. Accuracy: 0.9760
2. Precision: 0.9866
3. Recall: 0.9519
4. F1-score: 0.9689



## SVM

## Train svm model

### Subtask:
Train a Support Vector Machine (SVM) model on the TF-IDF transformed training data.


In [ ]:
from sklearn.svm import SVC

# Instantiate SVC model with a linear kernel
# 使用默认的线性核函数
svm_model = SVC(kernel='linear')

# Train the model
svm_model.fit(X_train_tfidf, y_train)

SVC(kernel='linear')

## Evaluate svm model

### Subtask:
Evaluate the performance of the trained SVM model on the test data using appropriate metrics.


**Reasoning**:
Calculate and print the evaluation metrics for the SVM model on the test data.



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test data
y_pred_svm = svm_model.predict(X_test_tfidf)

# Calculate evaluation metrics
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)

# Print the metrics
print(f"SVM Accuracy: {accuracy_svm:.4f}")
print(f"SVM Precision: {precision_svm:.4f}")
print(f"SVM Recall: {recall_svm:.4f}")
print(f"SVM F1-score: {f1_svm:.4f}")

SVM Accuracy: 0.9904
SVM Precision: 0.9903
SVM Recall: 0.9852
SVM F1-score: 0.9878


TF-IDF + SVM on sqli.csv
1. SVM Accuracy: 0.9759
2. SVM Precision: 1.0000
3. SVM Recall: 0.9069
4. SVM F1-score: 0.9512

TF-IDF + SVM on sqliv2.csv

1. SVM Accuracy: 0.9953
2. SVM Precision: 0.9978
3. SVM Recall: 0.9884
4. SVM F1-score: 0.9931

on Merged_data:
1. SVM Accuracy: 0.9904
2. SVM Precision: 0.9903
3. SVM Recall: 0.9852
4. SVM F1-score: 0.9878

## 1.3 Naive Bayes 朴素贝叶斯

## Train naive bayes model

### Subtask:
Train a Multinomial Naive Bayes model on the TF-IDF transformed training data.

使用 多项式朴素贝叶斯进行学习

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Instantiate MultinomialNB model
# 使用 多项式朴素贝叶斯
nb_model = MultinomialNB()

# Train the model
nb_model.fit(X_train_tfidf, y_train)

MultinomialNB()

**Reasoning**:
The variables X_train_tfidf and y_train were not defined in the current session, likely due to a kernel restart. Regenerate the code to split the data, apply TF-IDF vectorization, and then train the Naive Bayes model.



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

X = df['Sentence']
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer()

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

X_test_tfidf = tfidf_vectorizer.transform(X_test)

nb_model = MultinomialNB()

nb_model.fit(X_train_tfidf, y_train)

MultinomialNB()

**Reasoning**:
The variables X_train_tfidf and y_train were not defined in the current session, likely due to a kernel restart. Regenerate the code to split the data, apply TF-IDF vectorization, and then train the Naive Bayes model.

## Evaluate naive bayes model

### Subtask:
Evaluate the performance of the trained Naive Bayes model on the test data using appropriate metrics (accuracy, precision, recall, F1-score).


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test data
# 使用测试数据进行预测
y_pred_nb = nb_model.predict(X_test_tfidf)

# Calculate evaluation metrics
accuracy_nb = accuracy_score(y_test, y_pred_nb)
precision_nb = precision_score(y_test, y_pred_nb)
recall_nb = recall_score(y_test, y_pred_nb)
f1_nb = f1_score(y_test, y_pred_nb)

# Print the metrics
print(f"Naive Bayes Accuracy: {accuracy_nb:.4f}")
print(f"Naive Bayes Precision: {precision_nb:.4f}")
print(f"Naive Bayes Recall: {recall_nb:.4f}")
print(f"Naive Bayes F1-score: {f1_nb:.4f}")

Naive Bayes Accuracy: 0.9777
Naive Bayes Precision: 0.9656
Naive Bayes Recall: 0.9781
Naive Bayes F1-score: 0.9718


TF-IDF + Naive Bayes on sqli.csv:

* Naive Bayes Accuracy: 0.9722
* Naive Bayes Precision: 1.0000
* Naive Bayes Recall: 0.8922
* Naive Bayes F1-score: 0.9430

TF-IDF + Naive Bayes on sqliv2.csv

1. Naive Bayes Accuracy: 0.9827
2. Naive Bayes Precision: 0.9656
3. Naive Bayes Recall: 0.9850
4. Naive Bayes F1-score: 0.9752

on merged_data:
1. Naive Bayes Accuracy: 0.9777
2. Naive Bayes Precision: 0.9656
3. Naive Bayes Recall: 0.9781
4. Naive Bayes F1-score: 0.9718

# conclusion
以上，我们已经讨论了基本方法，有：
* 逻辑回归 Logistic regression
* SVM
* 朴素贝叶斯 naive Bayes

下面，我们可以使用一些更加复杂的方法。

# Word Embedding + neural network

词嵌入 + 神经网络



In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


# Instantiate Tokenizer
# 初始化 Tokenizer
# 只考虑频率最高的一万个词。OOV对应未出现的词
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>") # consider the top 10000 words

# Fit the tokenizer on the training data
# 在训练集上拟合 Tokenizer
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
# 将文本转换为序列
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Determine optimal max_length from your data (95th percentile to avoid outliers)
lengths = [len(seq) for seq in X_train_sequences]
max_length = int(np.percentile(lengths, 95))
print(f"Max length set to: {max_length} (95th percentile)")
print(f"Average length: {np.mean(lengths):.1f}")

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post', truncating='post')

print("Shape of training data after padding:", X_train_padded.shape)
print("Shape of testing data after padding:", X_test_padded.shape)

Max length set to: 29 (95th percentile)
Average length: 9.1
Shape of training data after padding: (45517, 29)
Shape of testing data after padding: (11380, 29)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Build the LSTM model
# 构建 LSTM 模型
# Build improved LSTM model with key enhancements
embedding_dim = 64  # Increased from 16 for better representation
model = Sequential([
    # Embedding layer
    Embedding(20000, embedding_dim, input_length=max_length),

    # Bidirectional LSTM captures context from both directions
    Bidirectional(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),

    # Second LSTM layer for deeper learning
    Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.2)),

    # Dense layer with dropout for regularization
    Dense(64, activation='relu'),
    Dropout(0.5),

    # Output layer
    Dense(1, activation='sigmoid')
])

# Compile the model
# 编译模型
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Print model summary
# 打印模型概况
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
# 训练模型
# Callbacks for better training
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=0.00001,
    verbose=1
)

# Train with validation split (not test set!)
history = model.fit(
    X_train_padded, y_train,
    epochs=10,  # Early stopping will prevent overfitting

    batch_size=64,  # Added explicit batch size
    validation_split=0.2,  # Use 20% of training data for validation
    callbacks=[early_stop, reduce_lr],
    verbose=2
)

Epoch 1/10
569/569 - 125s - 220ms/step - accuracy: 0.9749 - loss: 0.0736 - val_accuracy: 0.9973 - val_loss: 0.0124 - learning_rate: 1.0000e-03
Epoch 2/10
569/569 - 101s - 178ms/step - accuracy: 0.9964 - loss: 0.0184 - val_accuracy: 0.9978 - val_loss: 0.0118 - learning_rate: 1.0000e-03
Epoch 3/10
569/569 - 101s - 177ms/step - accuracy: 0.9974 - loss: 0.0131 - val_accuracy: 0.9963 - val_loss: 0.0214 - learning_rate: 1.0000e-03
Epoch 4/10
569/569 - 140s - 246ms/step - accuracy: 0.9977 - loss: 0.0105 - val_accuracy: 0.9962 - val_loss: 0.0151 - learning_rate: 1.0000e-03
Epoch 5/10

Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
569/569 - 144s - 252ms/step - accuracy: 0.9982 - loss: 0.0087 - val_accuracy: 0.9936 - val_loss: 0.0233 - learning_rate: 1.0000e-03
Epoch 6/10
569/569 - 141s - 248ms/step - accuracy: 0.9985 - loss: 0.0070 - val_accuracy: 0.9922 - val_loss: 0.0303 - learning_rate: 5.0000e-04
Epoch 7/10
569/569 - 142s - 249ms/step - accuracy: 0.9987 - loss:

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Make predictions on the test data
y_pred_probs = model.predict(X_test_padded)
y_pred = np.round(y_pred_probs).flatten()

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the metrics
print(f"LSTM Accuracy: {accuracy:.4f}")
print(f"LSTM Precision: {precision:.4f}")
print(f"LSTM Recall: {recall:.4f}")
print(f"LSTM F1-score: {f1:.4f}")

356/356 ━━━━━━━━━━━━━━━━━━━━ 13s 32ms/step
LSTM Accuracy: 0.9964
LSTM Precision: 0.9991
LSTM Recall: 0.9917
LSTM F1-score: 0.9954


LSTM 在merged_data 上面的数据
1. LSTM Accuracy: 0.9964
2. LSTM Precision: 0.9991
3. LSTM Recall: 0.9917
4. LSTM F1-score: 0.9954

# DistillBert



